In [1]:
import tensorflow as tf
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
## load the trained model , onehot encoder,label encoder ,and scaler pickle file
model = load_model('model.h5')

with open('label_encoder_gen.pkl','rb') as file:
    label_encoder_gen = pickle.load(file)

with open("onehot_encoder_geo.pkl",'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open("scaler.pkl",'rb') as file:
    scaler = pickle.load(file)

In [3]:
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' :3,
    'Balance' : 60000,
    'NumOfProducts' :2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [4]:
# conver the input data into dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
## perform label encoding on the Gender column of input_df
input_df['Gender'] = label_encoder_gen.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [6]:
## perform onehot encoding on Geography column
geo_encode = onehot_encoder_geo.transform([input_df["Geography"]]).toarray()
geo_encode_df = pd.DataFrame(geo_encode,columns = onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encode_df

d:\Ann_churn\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
## combine the geo_encode_df with input_df
input_df = pd.concat([input_df.drop('Geography',axis = 1),geo_encode_df],axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## scale the data
input_scaled = scaler.transform(input_df)

In [9]:
## model prediction
prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 120ms/step


In [10]:
prediction_proba = prediction[0][0]
prediction_proba

0.030660562

In [11]:
if prediction_proba>0.5 :
    print("customer is likely to churn.")
else:
    print("customer is not likely to churn.")

customer is not likely to churn.
